# Best model for pressure
The best model for pressure is the one with the following architecture:
- $7\,$ input parameters and $7$ output parameters
- $5\,$ hidden layers with $128$ neurons each with batch normalization
- `ReLU`,`Tanh` activation function
- `Adam` optimizer with learning rate $0.01$
- `ExponentialLR` scheduler with $\gamma=0.9995$
- `MSE` loss
- NO $L^1$ regularization
- $3000\,$ epochs

Make sure the folder `data` is in the principal directory.
Importing utils will load the data and the basis functions.

In [ ]:
# General setups and imports
from utils import *

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# Set the seed for reproducibility
if device=="cuda:0":
    torch.cuda.empty_cache()
    torch.cuda.manual_seed(42)
    torch.cuda.init()
    torch.cuda.empty_cache()
    
np.random.seed(42)
torch.manual_seed(42)

In [ ]:
# Normalization of input parameters in range [0,1]
maxs = np.array([8.0, 0.3, 0.5, 0.5, 0.5, 0.0])
mins = np.array([4.0, 0.1, -0.1, -0.5, -0.5, -0.3])
for i in range(params.shape[1]):
    params[:, i] = (params[:, i] - mins[i]) / (maxs[i] - mins[i])


# Shuffle the parameters
idx = np.random.permutation(params.shape[0])
params = params[idx]

# Expand pressure in time
pressure_time = solutions['pressure'] @ basis_time['pressure'].T

# Shuffle the pressure, the parameters are shuffled in the same way
pressure_time = pressure_time[idx]

# Split the data into training, validation and test set

# Training set: 80% of the data
# Validation set: 10% of the data
# Test set: 10% of the data

# Training set
params_train = params[:int(0.8 * len(params))]
pressure_time_train = pressure_time[:int(0.8 * len(params))]

# Validation set
params_val = params[int(0.8 * len(params)):int(0.9 * len(params))]
pressure_time_val = pressure_time[int(0.8 * len(params)):int(0.9 * len(params))]

# Test set
params_test = params[int(0.9 * len(params)):]
pressure_time_test = pressure_time[int(0.9 * len(params)):]


# Treat time as a parameter: add it to the parameter list
# u1 u2 u3 u4 u5 u6 t 
times = np.linspace(0, 1, 300)

#sample all the times with times[:] 
#sample every 5 timesteps with times[::5] 
times_test= times[:]
times_train= times[:]
times_val= times[:]

# generate a matrix with parameters for each time step for training, validation and test set
# add time to the parameter vector
params_time_train = np.repeat(params_train, len(times_train), axis=0)
params_time_train = np.hstack((params_time_train, np.tile(times_train, len(params_train)).reshape(-1, 1)))

params_time_val = np.repeat(params_val, len(times_val), axis=0)
params_time_val = np.hstack((params_time_val, np.tile(times_val, len(params_val)).reshape(-1, 1)))

params_time_test = np.repeat(params_test, len(times_test), axis=0)
params_time_test = np.hstack((params_time_test, np.tile(times_test, len(params_test)).reshape(-1, 1)))

# if times[:] put vel_time_test[:, :, :]
# if times[::5] put vel_time_test[:, :, ::5] 
pressure_model_test= pressure_time_test[:, :, :]
pressure_model_train= pressure_time_train[:, :, :]
pressure_model_val= pressure_time_val[:, :, :]

# Reshape the data to have the form (number of samples, number of parameters, number of time steps)
pressure_model_train = pressure_model_train.transpose(0, 2, 1).reshape((pressure_model_train.shape[0] * len(times_train)), 7)
pressure_model_val = pressure_model_val.transpose(0, 2, 1).reshape((pressure_model_val.shape[0] * len(times_val)), 7)
pressure_model_test = pressure_model_test.transpose(0, 2, 1).reshape((pressure_model_test.shape[0] * len(times_test)), 7)

# Take the SV coefficients of the pressure and normalize them
sv_space_pressure = sv_space['pressure']
sv_space_pressure = sv_space_pressure / np.sum(sv_space_pressure)


# Convert to tensor
params_time_train = torch.tensor(params_time_train, dtype=torch.float32).to(device)
params_time_val = torch.tensor(params_time_val, dtype=torch.float32).to(device)
params_time_test = torch.tensor(params_time_test, dtype=torch.float32).to(device)

pressure_model_train = torch.tensor(pressure_model_train, dtype=torch.float32).to(device)
pressure_model_val = torch.tensor(pressure_model_val, dtype=torch.float32).to(device)
pressure_model_test = torch.tensor(pressure_model_test, dtype=torch.float32).to(device)

sv_space_pressure = torch.tensor(sv_space_pressure, dtype=torch.float32).to(device)
sv_space_pressure = sv_space_pressure.reshape(7, 1)

# Possible logarithmic transformation of the data
#pressure_model_train = torch.log(torch.abs(pressure_model_train) + 1) * torch.sign(pressure_model_train)
#pressure_model_val = torch.log(torch.abs(pressure_model_val) + 1) * torch.sign(pressure_model_val)

In [ ]:
# Define the network
class Net(torch.nn.Module):
    # 7 input parameters, corresponding to: u1 u2 u3 u4 u5 u6 t
    # 7 output parameters, corresponding to the POD (reduced) coefficients of the pressure solution
    # 5 hidden layers with 128 neurons each with batch normalization
    # ReLU,Tanh activation function
    def __init__(self, input_size, hidden_size, output_size):
        super(Net, self).__init__()
        self.fc1 = torch.nn.Linear(input_size, hidden_size)
        self.F1 = torch.nn.ReLU()
        self.batch_norm1 = torch.nn.BatchNorm1d(hidden_size)
        self.fc2 = torch.nn.Linear(hidden_size, hidden_size)
        self.F2 = torch.nn.Tanh()
        self.batch_norm2 = torch.nn.BatchNorm1d(hidden_size)
        self.fc3 = torch.nn.Linear(hidden_size, hidden_size)
        self.F3 = torch.nn.ReLU()
        self.batch_norm3 = torch.nn.BatchNorm1d(hidden_size)
        self.fc4 = torch.nn.Linear(hidden_size, hidden_size)
        self.F4 = torch.nn.Tanh()
        self.batch_norm4 = torch.nn.BatchNorm1d(hidden_size)
        self.fc5 = torch.nn.Linear(hidden_size, hidden_size)
        self.F5 = torch.nn.ReLU()
        self.batch_norm5 = torch.nn.BatchNorm1d(hidden_size)
        self.fc6 = torch.nn.Linear(hidden_size, output_size)


    def forward(self, x):
        x = self.F1(self.fc1(x))
        x = self.batch_norm1(x)
        x = self.F2(self.fc2(x))
        x = self.batch_norm2(x)
        x = self.F3(self.fc3(x))
        x = self.batch_norm3(x)
        x = self.F4(self.fc4(x))
        x = self.batch_norm4(x)
        x = self.F5(self.fc5(x))
        x = self.batch_norm5(x)
        return self.fc6(x)


# Define the parameters of the network
input_size = 7
hidden_size = 128 # constant for all the layers
output_size = 7

# Create the network
net = Net(input_size=input_size, hidden_size=hidden_size, output_size=output_size).to(device)

# Define the loss function, weighting the pressure loss with the sqrt sv coefficients
#def loss_fn(y_pred, y_true):
#    return torch.mean(torch.mm((y_pred - y_true) ** 2, (sv_space_pressure)**(1/2)))

# Define the loss function as the MSE loss
loss_fn = torch.nn.MSELoss()

learning_rate = .01 # Starting learning rate
#optimizer = torch.optim.AdamW(net.parameters(), lr=learning_rate)
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)

gamma = 0.9995  # The factor by which the learning rate will be multiplied at each epoch

# Create the ExponentialLR scheduler
scheduler = torch.optim.lr_scheduler.ExponentialLR(optimizer, gamma=gamma)

# L1 regularization if needed
l1_lambda = 0 # Set to 0 if you do not want to use L1 regularization

if l1_lambda > 0:
    print("L1 regularization enabled")
    nweights = 0
    for name,weights in net.named_parameters():
        if 'bias' not in name:
            nweights = nweights + weights.numel()
    print(f'Total number of weights in the model = {nweights}')
else:
    print("L1 regularization disabled")


# Save the loss functions for each iteration
losses_train = []
losses_val = []

# Save the absolute errors every 100 iteration (for memory reasons)
err_val = []
err_train = []

In [ ]:
# Load the model and the losses if present, run this cell if you do not want to train or if you want to continue a previous training

if os.path.exists("losses_val_pressure.npy"):
   losses_train = np.load("losses_val_pressure.npy").tolist()
if os.path.exists("losses_train_pressure.npy"):
   losses_val = np.load("losses_train_pressure.npy").tolist()

if os.path.exists("model_pressure.pt"):
    net.load_state_dict(torch.load("model_pressure.pt"))


In [ ]:
""" Train the network"""

for t in range(3000):
    net.train()

    # Forward pass: compute predicted y by passing x to the model.
    y_pred = net(params_time_train).to(device)

    # if you want to compute the training error every 100 iterations
    """
    if t %100 ==0:
      # compute the mean across the time steps for each simulation
      mean_pres=torch.mean(torch.norm(pressure_model_train, dim=1).reshape(-1, len(times_train)), dim=1)
      # repeat the mean for every time step
      mean_pres = torch.repeat(mean_pres, [len(times_train),1]).reshape(1, -1)
      # compute the absolute error for each simulation 
      abs_error = torch.norm(y_pred - pressure_model_train, dim=1)/mean_pres
      
      err_train.append(torch.mean(abs_error).to("cpu").detach().numpy())
    """


    # Compute train loss
    loss_train = loss_fn(y_pred, pressure_model_train)

    losses_train.append(loss_train.item())

    # L1 regularization if needed
    
    if l1_lambda > 0:
        # Compute L1 term
        L1_term = torch.tensor(0., requires_grad=True).to(device)
        for name, weights in net.named_parameters():
            if 'bias' not in name:
                    weights_sum = torch.sum(torch.abs(weights))
                    L1_term = L1_term + weights_sum
        L1_term = L1_term / nweights
        # Regularize loss using L1 regularization
        loss_train = loss_train + L1_term * l1_lambda

    # Before the backward pass, use the optimizer object to zero all the gradients for the variables it will update (which are the learnable weights of the model)
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model parameters
    loss_train.backward()

    # Calling the step function on an Optimizer makes an update to its parameters
    optimizer.step()
    
    # Update the learning rate
    scheduler.step()

    # Validation
    net.eval()
    with torch.no_grad():
        y_pred_val = net(params_time_val).to(device)
        loss_val = loss_fn(y_pred_val, pressure_model_val)
        
        # if you want to  compute the relative error every 100 iterations
        """
        if t%100==0:
          # compute the mean across the time steps for each simulation
          mean_pres=torch.mean(torch.norm(pressure_model_val, dim=1).reshape(-1, len(times_val)), dim=1)
          # repeat the mean for every time step
          mean_pres = torch.repeat(mean_pres, [len(times_val),1]).reshape(1, -1)
          # compute the absolute error for each simulation
          abs_error = torch.norm(y_pred_val - pressure_model_val, dim=1)/mean_pres
          print("Validation, " , torch.mean(abs_error))

          err_val.append(torch.mean(abs_error).to("cpu").detach().numpy())
        """
    losses_val.append(loss_val.item()) 
    
    if t % 100 == 0:
        print("Epoch: ", t, "Train Loss: ", loss_train.item(),", Validation Loss: ", loss_val.item())
    

In [ ]:
# Save the model and losses, run this cell if you want to save the model and the losses
torch.save(net.state_dict(), "model_pressure.pt")
np.save("losses_val_pressure.npy", losses_val)
np.save("losses_train_pressure.npy", losses_train)

In [ ]:
""" Test the network"""

# Evaluate the model on the test set
y_pred = net(params_time_test)
y_pred_numpy=y_pred.to("cpu")

# convert the output of the NN to numpy array
y_pred_numpy = y_pred_numpy.detach().numpy()

# if you have considered the logarithmic transformation you have to perform the inverse transformation
#y_pred_numpy=np.exp(y_pred_numpy)-1

# convert the pressure_model_test to numpy
pressure_model_test_numpy=pressure_model_test.to("cpu")
pressure_model_test_numpy=pressure_model_test_numpy.detach().numpy()

# Compute and print loss.
loss_t = torch.nn.MSELoss()(y_pred, pressure_model_test)

print("Test loss: ", loss_t.item())

# Compute the relative error

rel_error = np.linalg.norm(y_pred_numpy - pressure_model_test_numpy, axis=1) / np.linalg.norm(pressure_model_test_numpy, axis=1)

#Compute average values
mean_pres=np.mean(np.linalg.norm(pressure_model_test_numpy, axis=1).reshape(-1, len(times_test)), axis=1)
# repeat the mean for every time step
mean_pres = np.repeat(mean_pres, len(times_test)).reshape(1, -1)
#Compute absolute error rescaled by average values within each simulation
abs_error = np.linalg.norm(y_pred_numpy - pressure_model_test_numpy, axis=1)/mean_pres
abs_error = abs_error.T

#np.save("abs_error_pressure.npy", abs_error)
print("Relative error: ", np.mean(rel_error))
print("Absolute error: ", np.mean(abs_error))


In [ ]:
# Plot the losses function

plt.plot(losses_train, label="train")
plt.plot(losses_val, label="val")
plt.legend()
plt.title("Losses for pressure")
plt.savefig("losses_pressure.png")
plt.show()

In [ ]:
# Plot the train and validation error across the epochs, if computed

plt.plot(err_train, label="train")
plt.plot(err_val, label="val")

plt.legend()

x_ticks_locations = np.arange(0, 40, 5)
xs=np.arange(0,4000, 500)
x_ticks_labels = [str(int(label)) for label in xs]
plt.xticks(x_ticks_locations, x_ticks_labels)

plt.xlabel("epochs")
plt.title("Absolute error pressure")
#plt.savefig("absolute_error_pressure.png")
plt.show()


In [ ]:
"""Time-space error"""

# Reshape and transpose the data to have the form (number of samples, number of space coefficients, number of time steps)
Test = np.zeros((len(params_test), len(times_test), 7))
for i in range(len(params_test)):
    Test[i, :, :] = y_pred_numpy[i * len(times_test):(i + 1) * len(times_test), :]
Test = Test.transpose(0, 2, 1)

# Take the true solution
TrUE = pressure_time_test[:, :, :]

# compute the space norm of the error
norm_s = np.linalg.norm(Test - TrUE, axis=1)

# compute the time norm of norm_s to obtain the time-space norm of the error
norm_st = np.linalg.norm(norm_s, axis=1)

# compute the space norm of the true solution
norm_s_TRUE = np.linalg.norm(TrUE, axis=1)

# compute the time norm to obtain the time-space norm of the true solution
norm_st_TRUE = np.linalg.norm(norm_s_TRUE, axis=1)

# compute the final error
err_vec = np.divide(norm_st, norm_st_TRUE)

# print the mean and the standard deviation of the error
print("Mean error=", np.mean(err_vec))
print("Standard deviation=", np.std(err_vec))

### Visualize the solution
After training the network, you can visualize the solution in space-time.
Open the group of generated files with `Paraview`.


In [ ]:
""" Visualize the solution of the pressure in space-time."""
# Save predicted solution in a vtk file for visualization in Paraview
params_for_simulation=params_time_test[:301, :]

sol_pred = net(params_for_simulation).to(device)
sol_pred=sol_pred.to("cpu")
sol_pred=sol_pred.detach().numpy()

params = np.load(os.path.join('../data/RB_data', 'parameters.npy'))
params = np.delete(params, 2, axis=1)

# create a new folder for the solutions
os.makedirs('solutions_pred', exist_ok=True)

# read the mesh and create cur_idxs
idxs = compute_matching_idxs()
mesh = read_vtk(os.path.join('../data/geometries', 'tube_1x4.vtu'))
fom_solution = dict()
field = 'pressure'

cur_idxs = np.hstack([idxs + k * (Nh_space[field] // 1) for k in range(1)])

# expand the solution in space
fom_solution[field] = basis_space['pressure'].dot(sol_pred.T)[cur_idxs]
# fix the time step for the visualization
step_t = 5

# add the solution to the mesh and write the vtk file for visualization in Paraview
for cnt_t in range(0, Nh_time['pressure'], step_t):
    cur_fom_solution = np.reshape(fom_solution[field][:, cnt_t], (1, -1)).T
    mesh = add_array(mesh, cur_fom_solution, field)

    write_vtk(mesh, os.path.join('solutions_pred', f"pressure_{0}_{cnt_t}" + '.vtu'))